# Tutorial of scMDC on 10X PBMC CITE-seq data

This tutorial was implemented on MacBook pro 2018 with CPU

1. Build the scMDC model and load the data

In [1]:
from time import time
import math, os
from sklearn import metrics
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from scMDC import scMultiCluster
import numpy as np
import collections
import h5py
import scanpy as sc
from preprocess import read_dataset, normalize
from utils import cluster_acc, GetCluster


data_mat = h5py.File('10XPBMC_filtered_1000G.H5')
x1 = np.array(data_mat['X1'])
x2 = np.array(data_mat['X2'])
y = np.array(data_mat['Y'])
data_mat.close()

# preprocessing CITE-seq read counts matrix
adata1 = sc.AnnData(x1)
adata1.obs['Group'] = y

adata1 = read_dataset(adata1,
                    transpose=False,
                    test_split=False,
                    copy=True)

adata1 = normalize(adata1,
                    size_factors=True,
                    normalize_input=True,
                    logtrans_input=True)

adata2 = sc.AnnData(x2)
adata2.obs['Group'] = y
adata2 = read_dataset(adata2,
                    transpose=False,
                    test_split=False,
                    copy=True)

adata2 = normalize(adata2,
                    size_factors=True,
                    normalize_input=True,
                    logtrans_input=True)

input_size1 = adata1.n_vars
input_size2 = adata2.n_vars

print(adata1.X.shape)
print(adata2.raw.X.shape)
print(x2.shape)
print(adata2.X.shape)
print(y.shape)

encodeLayer1 = list(map(int, [64,32,12]))
decodeLayer1 = encodeLayer1[::-1]
encodeLayer2 = list(map(int, [8]))
if len(encodeLayer2) >1:
    decodeLayer2 = encodeLayer2[::-1]
else:
    decodeLayer2 = encodeLayer2
    
encodeLayer3 = list(map(int, [64,16]))
if len(encodeLayer3) >1:
    decodeLayer3 = encodeLayer3[::-1]
else:
    decodeLayer3 = encodeLayer3
    
model = scMultiCluster(input_dim1=input_size1, input_dim2=input_size2,
                    zencode_dim=encodeLayer3, zdecode_dim=decodeLayer3, 
                    encodeLayer1=encodeLayer1, decodeLayer1=decodeLayer1, encodeLayer2=encodeLayer2, decodeLayer2=decodeLayer2,
                    sigma1=2.5, sigma2=0, gamma1=0.1, gamma2=0.1, gamma3=.0001, cutoff = 0.5)

print(str(model))

/Users/tiantian/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


### Autoencoder: Successfully preprocessed 1000 genes and 6661 cells.
### Autoencoder: Successfully preprocessed 14 genes and 6661 cells.
(6661, 1000)
(6661, 14)
(6661, 14)
(6661, 14)
(6661,)
scMultiCluster(
  (encoder1): Sequential(
    (0): Linear(in_features=1000, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=12, bias=True)
  )
  (decoder1): Sequential(
    (0): Linear(in_features=12, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
  )
  (encoder2): Sequential(
    (0): Linear(in_features=14, out_features=8, bias=True)
  )
  (decoder2): Sequential()
  (latent_enc): Sequential(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=16, bias=True)
  )
  (latent_dec): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=

2. Pretrainging state

In [2]:
#pretraing stage
t0 = time()

model.pretrain_autoencoder(X1=adata1.X, X_raw1=adata1.raw.X, sf1=adata1.obs.size_factors, X2=adata2.X, X_raw2=adata2.raw.X, sf2=adata2.obs.size_factors, batch_size=256, epochs=400, ae_weights='AE_weights_1.pth.tar')

print('Pretraining time: %d seconds.' % int(time() - t0))

Pretraining stage
Pretrain epoch [1/1], ZINB loss:0.7685, NB loss:244.6999
Pretrain epoch [2/1], ZINB loss:0.7862, NB loss:237.7955
Pretrain epoch [3/1], ZINB loss:0.7567, NB loss:231.7954
Pretrain epoch [4/1], ZINB loss:0.6953, NB loss:220.0811
Pretrain epoch [5/1], ZINB loss:0.6515, NB loss:213.8475
Pretrain epoch [6/1], ZINB loss:0.6197, NB loss:208.9973
Pretrain epoch [7/1], ZINB loss:0.5906, NB loss:199.4105
Pretrain epoch [8/1], ZINB loss:0.5331, NB loss:189.3931
Pretrain epoch [9/1], ZINB loss:0.5357, NB loss:187.7750
Pretrain epoch [10/1], ZINB loss:0.5405, NB loss:179.2274
Pretrain epoch [11/1], ZINB loss:0.5366, NB loss:173.3947
Pretrain epoch [12/1], ZINB loss:0.5040, NB loss:169.4115
Pretrain epoch [13/1], ZINB loss:0.5066, NB loss:165.8712
Pretrain epoch [14/1], ZINB loss:0.4927, NB loss:156.6874
Pretrain epoch [15/1], ZINB loss:0.4952, NB loss:159.2221
Pretrain epoch [16/1], ZINB loss:0.4942, NB loss:151.8865
Pretrain epoch [17/1], ZINB loss:0.4868, NB loss:149.1227
Pretr

Pretrain epoch [10/6], ZINB loss:0.4229, NB loss:11.5235
Pretrain epoch [11/6], ZINB loss:0.4191, NB loss:11.5939
Pretrain epoch [12/6], ZINB loss:0.4123, NB loss:11.5202
Pretrain epoch [13/6], ZINB loss:0.4213, NB loss:11.1892
Pretrain epoch [14/6], ZINB loss:0.4002, NB loss:11.3524
Pretrain epoch [15/6], ZINB loss:0.4062, NB loss:11.1018
Pretrain epoch [16/6], ZINB loss:0.4213, NB loss:11.2628
Pretrain epoch [17/6], ZINB loss:0.3971, NB loss:10.9121
Pretrain epoch [18/6], ZINB loss:0.4123, NB loss:10.7550
Pretrain epoch [19/6], ZINB loss:0.4089, NB loss:10.6411
Pretrain epoch [20/6], ZINB loss:0.4258, NB loss:10.6300
Pretrain epoch [21/6], ZINB loss:0.3789, NB loss:10.8499
Pretrain epoch [22/6], ZINB loss:0.4172, NB loss:10.4883
Pretrain epoch [23/6], ZINB loss:0.4080, NB loss:10.5355
Pretrain epoch [24/6], ZINB loss:0.4435, NB loss:10.8180
Pretrain epoch [25/6], ZINB loss:0.3984, NB loss:10.2904
Pretrain epoch [26/6], ZINB loss:0.3886, NB loss:10.5180
Pretrain epoch [27/6], ZINB los

Pretrain epoch [22/11], ZINB loss:0.4008, NB loss:7.4223
Pretrain epoch [23/11], ZINB loss:0.4104, NB loss:7.4230
Pretrain epoch [24/11], ZINB loss:0.3976, NB loss:7.3854
Pretrain epoch [25/11], ZINB loss:0.3898, NB loss:7.3617
Pretrain epoch [26/11], ZINB loss:0.4104, NB loss:7.3753
Pretrain epoch [27/11], ZINB loss:0.4379, NB loss:7.7270
Pretrain epoch [1/12], ZINB loss:0.4142, NB loss:7.2982
Pretrain epoch [2/12], ZINB loss:0.4221, NB loss:7.3532
Pretrain epoch [3/12], ZINB loss:0.3929, NB loss:7.3227
Pretrain epoch [4/12], ZINB loss:0.4079, NB loss:7.2940
Pretrain epoch [5/12], ZINB loss:0.4028, NB loss:7.3548
Pretrain epoch [6/12], ZINB loss:0.3911, NB loss:7.3018
Pretrain epoch [7/12], ZINB loss:0.4070, NB loss:7.3199
Pretrain epoch [8/12], ZINB loss:0.3897, NB loss:7.3123
Pretrain epoch [9/12], ZINB loss:0.4108, NB loss:7.3426
Pretrain epoch [10/12], ZINB loss:0.3879, NB loss:7.2869
Pretrain epoch [11/12], ZINB loss:0.4183, NB loss:7.3355
Pretrain epoch [12/12], ZINB loss:0.3913

Pretrain epoch [6/17], ZINB loss:0.3877, NB loss:6.6217
Pretrain epoch [7/17], ZINB loss:0.4047, NB loss:6.5931
Pretrain epoch [8/17], ZINB loss:0.3960, NB loss:6.6003
Pretrain epoch [9/17], ZINB loss:0.3904, NB loss:6.6170
Pretrain epoch [10/17], ZINB loss:0.4088, NB loss:6.6186
Pretrain epoch [11/17], ZINB loss:0.3899, NB loss:6.5849
Pretrain epoch [12/17], ZINB loss:0.4095, NB loss:6.6228
Pretrain epoch [13/17], ZINB loss:0.3783, NB loss:6.5799
Pretrain epoch [14/17], ZINB loss:0.4109, NB loss:6.6175
Pretrain epoch [15/17], ZINB loss:0.4005, NB loss:6.5944
Pretrain epoch [16/17], ZINB loss:0.4013, NB loss:6.5919
Pretrain epoch [17/17], ZINB loss:0.3848, NB loss:6.5743
Pretrain epoch [18/17], ZINB loss:0.4195, NB loss:6.6341
Pretrain epoch [19/17], ZINB loss:0.4056, NB loss:6.5652
Pretrain epoch [20/17], ZINB loss:0.4017, NB loss:6.5480
Pretrain epoch [21/17], ZINB loss:0.4113, NB loss:6.5292
Pretrain epoch [22/17], ZINB loss:0.3907, NB loss:6.6087
Pretrain epoch [23/17], ZINB loss:0

Pretrain epoch [16/22], ZINB loss:0.3976, NB loss:6.2902
Pretrain epoch [17/22], ZINB loss:0.3969, NB loss:6.2999
Pretrain epoch [18/22], ZINB loss:0.4102, NB loss:6.2948
Pretrain epoch [19/22], ZINB loss:0.3839, NB loss:6.3357
Pretrain epoch [20/22], ZINB loss:0.3993, NB loss:6.2622
Pretrain epoch [21/22], ZINB loss:0.4110, NB loss:6.2831
Pretrain epoch [22/22], ZINB loss:0.4075, NB loss:6.2900
Pretrain epoch [23/22], ZINB loss:0.4030, NB loss:6.2727
Pretrain epoch [24/22], ZINB loss:0.3958, NB loss:6.2472
Pretrain epoch [25/22], ZINB loss:0.3978, NB loss:6.2772
Pretrain epoch [26/22], ZINB loss:0.3860, NB loss:6.2362
Pretrain epoch [27/22], ZINB loss:0.3093, NB loss:6.5036
Pretrain epoch [1/23], ZINB loss:0.3908, NB loss:6.2844
Pretrain epoch [2/23], ZINB loss:0.3862, NB loss:6.2527
Pretrain epoch [3/23], ZINB loss:0.4026, NB loss:6.3013
Pretrain epoch [4/23], ZINB loss:0.3999, NB loss:6.2755
Pretrain epoch [5/23], ZINB loss:0.4022, NB loss:6.2317
Pretrain epoch [6/23], ZINB loss:0.3

...

Pretrain epoch [11/387], ZINB loss:0.4018, NB loss:4.4861, latent MSE loss:0.00033515, KL loss:0.00077458
Pretrain epoch [12/387], ZINB loss:0.3954, NB loss:4.4778, latent MSE loss:0.00028918, KL loss:0.00090135
Pretrain epoch [13/387], ZINB loss:0.3838, NB loss:4.4974, latent MSE loss:0.00027357, KL loss:0.00075640
Pretrain epoch [14/387], ZINB loss:0.3872, NB loss:4.4329, latent MSE loss:0.00029223, KL loss:0.00073909
Pretrain epoch [15/387], ZINB loss:0.3914, NB loss:4.5178, latent MSE loss:0.00030429, KL loss:0.00073527
Pretrain epoch [16/387], ZINB loss:0.3994, NB loss:4.4837, latent MSE loss:0.00026220, KL loss:0.00079508
Pretrain epoch [17/387], ZINB loss:0.3798, NB loss:4.4954, latent MSE loss:0.00028466, KL loss:0.00069666
Pretrain epoch [18/387], ZINB loss:0.3916, NB loss:4.4545, latent MSE loss:0.00023136, KL loss:0.00069886
Pretrain epoch [19/387], ZINB loss:0.3815, NB loss:4.4617, latent MSE loss:0.00022676, KL loss:0.00072047
Pretrain epoch [20/387], ZINB loss:0.4084, NB 

Pretrain epoch [9/390], ZINB loss:0.3834, NB loss:4.4895, latent MSE loss:0.00021450, KL loss:0.00078818
Pretrain epoch [10/390], ZINB loss:0.4099, NB loss:4.4413, latent MSE loss:0.00028444, KL loss:0.00080093
Pretrain epoch [11/390], ZINB loss:0.4089, NB loss:4.4847, latent MSE loss:0.00030687, KL loss:0.00098911
Pretrain epoch [12/390], ZINB loss:0.3891, NB loss:4.4753, latent MSE loss:0.00023073, KL loss:0.00074354
Pretrain epoch [13/390], ZINB loss:0.4066, NB loss:4.4292, latent MSE loss:0.00024257, KL loss:0.00083262
Pretrain epoch [14/390], ZINB loss:0.3992, NB loss:4.4969, latent MSE loss:0.00027434, KL loss:0.00116926
Pretrain epoch [15/390], ZINB loss:0.3726, NB loss:4.4353, latent MSE loss:0.00025704, KL loss:0.00072312
Pretrain epoch [16/390], ZINB loss:0.3939, NB loss:4.4964, latent MSE loss:0.00021355, KL loss:0.00077183
Pretrain epoch [17/390], ZINB loss:0.3814, NB loss:4.5375, latent MSE loss:0.00023977, KL loss:0.00077935
Pretrain epoch [18/390], ZINB loss:0.4024, NB l

Pretrain epoch [7/393], ZINB loss:0.3864, NB loss:4.4830, latent MSE loss:0.00078177, KL loss:0.00081906
Pretrain epoch [8/393], ZINB loss:0.3922, NB loss:4.4269, latent MSE loss:0.00072826, KL loss:0.00119887
Pretrain epoch [9/393], ZINB loss:0.3687, NB loss:4.5159, latent MSE loss:0.00069540, KL loss:0.00078109
Pretrain epoch [10/393], ZINB loss:0.3945, NB loss:4.4662, latent MSE loss:0.00074935, KL loss:0.00084440
Pretrain epoch [11/393], ZINB loss:0.3902, NB loss:4.4382, latent MSE loss:0.00079315, KL loss:0.00084853
Pretrain epoch [12/393], ZINB loss:0.3877, NB loss:4.4860, latent MSE loss:0.00070466, KL loss:0.00092271
Pretrain epoch [13/393], ZINB loss:0.4029, NB loss:4.4224, latent MSE loss:0.00067502, KL loss:0.00105961
Pretrain epoch [14/393], ZINB loss:0.4106, NB loss:4.4409, latent MSE loss:0.00067609, KL loss:0.00085322
Pretrain epoch [15/393], ZINB loss:0.4071, NB loss:4.4879, latent MSE loss:0.00067822, KL loss:0.00089524
Pretrain epoch [16/393], ZINB loss:0.4114, NB los

Pretrain epoch [5/396], ZINB loss:0.3817, NB loss:4.4713, latent MSE loss:0.00030490, KL loss:0.00108132
Pretrain epoch [6/396], ZINB loss:0.4015, NB loss:4.4533, latent MSE loss:0.00025933, KL loss:0.00076624
Pretrain epoch [7/396], ZINB loss:0.4057, NB loss:4.4919, latent MSE loss:0.00034363, KL loss:0.00089915
Pretrain epoch [8/396], ZINB loss:0.3939, NB loss:4.4323, latent MSE loss:0.00020799, KL loss:0.00074393
Pretrain epoch [9/396], ZINB loss:0.3906, NB loss:4.4038, latent MSE loss:0.00029061, KL loss:0.00071624
Pretrain epoch [10/396], ZINB loss:0.4025, NB loss:4.4770, latent MSE loss:0.00020271, KL loss:0.00133975
Pretrain epoch [11/396], ZINB loss:0.3986, NB loss:4.4698, latent MSE loss:0.00025373, KL loss:0.00071064
Pretrain epoch [12/396], ZINB loss:0.3945, NB loss:4.5209, latent MSE loss:0.00019901, KL loss:0.00096835
Pretrain epoch [13/396], ZINB loss:0.3813, NB loss:4.5383, latent MSE loss:0.00024891, KL loss:0.00084810
Pretrain epoch [14/396], ZINB loss:0.3999, NB loss:

Pretrain epoch [3/399], ZINB loss:0.4022, NB loss:4.4887, latent MSE loss:0.00041145, KL loss:0.00073374
Pretrain epoch [4/399], ZINB loss:0.4099, NB loss:4.4659, latent MSE loss:0.00037325, KL loss:0.00078762
Pretrain epoch [5/399], ZINB loss:0.4104, NB loss:4.4534, latent MSE loss:0.00041327, KL loss:0.00082999
Pretrain epoch [6/399], ZINB loss:0.3887, NB loss:4.5102, latent MSE loss:0.00041612, KL loss:0.00066837
Pretrain epoch [7/399], ZINB loss:0.4036, NB loss:4.5286, latent MSE loss:0.00033993, KL loss:0.00114525
Pretrain epoch [8/399], ZINB loss:0.3800, NB loss:4.4345, latent MSE loss:0.00033332, KL loss:0.00068893
Pretrain epoch [9/399], ZINB loss:0.3921, NB loss:4.3842, latent MSE loss:0.00047299, KL loss:0.00103625
Pretrain epoch [10/399], ZINB loss:0.3974, NB loss:4.4292, latent MSE loss:0.00029465, KL loss:0.00081653
Pretrain epoch [11/399], ZINB loss:0.4012, NB loss:4.4544, latent MSE loss:0.00036789, KL loss:0.00090803
Pretrain epoch [12/399], ZINB loss:0.3955, NB loss:4.

3. Estimate the number of clusters on latent features by Louvain algorithm

In [3]:
#estimate k
latent = model.encodeBatch(torch.tensor(adata1.X), torch.tensor(adata2.X)).cpu().numpy()
n_clusters = GetCluster(latent, res=0.2, n=30)
print("Estimated number of clusters:", n_clusters)

Estimated n_clusters is:  5
Estimated number of clusters: 5


4. Clustering stage

In [4]:
y_pred, _, _, _, _ = model.fit(X1=adata1.X, X_raw1=adata1.raw.X, sf1=adata1.obs.size_factors, X2=adata2.X, X_raw2=adata2.raw.X, sf2=adata2.obs.size_factors, y=y, n_clusters=n_clusters, batch_size=256, num_epochs=2000, update_interval=1, tol=0.001, lr=1., save_dir='results/')
print('Total time: %d seconds.' % int(time() - t0))

acc = np.round(cluster_acc(y, y_pred), 5)
nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
print('Final: ACC= %.4f, NMI= %.4f, ARI= %.4f' % (acc, nmi, ari))

Clustering stage
Initializing cluster centers with kmeans.


/Users/tiantian/miniconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/Users/tiantian/miniconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Initializing k-means: ACC= 0.9707, NMI= 0.9194, ARI= 0.9353


/Users/tiantian/miniconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Clustering   1: ACC= 0.9707, NMI= 0.9194, ARI= 0.9353
#Epoch   1: Total: 15.7939 Clustering Loss: 0.02671068 ZINB Loss: 0.3940 NB Loss: 4.4879 Latent MSE Loss: 0.1198 KL Loss: 10.7654
Clustering   2: ACC= 0.9710, NMI= 0.9199, ARI= 0.9360
delta_label  0.0006005104338687885 < tol  0.001
Reach tolerance threshold. Stopping training.
Total time: 1521 seconds.
Final: ACC= 0.9710, NMI= 0.9199, ARI= 0.9360


/Users/tiantian/miniconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/Users/tiantian/miniconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
